# Interview Analysis Notebook

This notebook is set up to load the provided CSV files and explore them systematically.

In [31]:
# --- Colab bootstrap (safe to re-run; never nests paths) ---
REPO_SLUG  = "monkeymoves/nesta"       # <user>/<repo>
TARGET_DIR = "/content/nesta"          # absolute path for the repo on Colab

try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

import os, sys, subprocess

if IN_COLAB:
    # Clone exactly once
    if not os.path.isdir(TARGET_DIR):
        subprocess.run(["git", "clone", f"https://github.com/{REPO_SLUG}.git", TARGET_DIR], check=True)

    # Chdir only if we're not already there
    if os.getcwd() != TARGET_DIR:
        os.chdir(TARGET_DIR)

    # Ensure the repo root is on sys.path (for any local imports if you add them later)
    if TARGET_DIR not in sys.path:
        sys.path.insert(0, TARGET_DIR)

# Quick sanity print
print("CWD:", os.getcwd())

CWD: /Users/lukemaggs/Desktop/Nesta_Interview_LM/notebooks


In [39]:
# === Config (works local + Colab) ===
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":  # if running from /notebooks, go up one level
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_RAW       = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

FILENAMES = [
    "lfs_monthly_variables.csv",
    "lfs_quarterly_variables.csv",
    "qual_survey_responses.csv",
    "variable_names.csv",
]

print("CWD:", Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_RAW:", DATA_RAW)
print("Files:", FILENAMES)

CWD: /Users/lukemaggs/Desktop/Nesta_Interview_LM/notebooks
PROJECT_ROOT: /Users/lukemaggs/Desktop/Nesta_Interview_LM
DATA_RAW: /Users/lukemaggs/Desktop/Nesta_Interview_LM/data/raw
Files: ['lfs_monthly_variables.csv', 'lfs_quarterly_variables.csv', 'qual_survey_responses.csv', 'variable_names.csv']


In [40]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

def raw_path(*parts): return DATA_RAW.joinpath(*parts)

def read_csv_local(filename: str, **kwargs) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1"):
        try:
            return pd.read_csv(raw_path(filename), encoding=enc, **kwargs)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(raw_path(filename), encoding="latin-1", encoding_errors="replace", **kwargs)

# Load
dfs = {}
for fn in FILENAMES:
    p = raw_path(fn)
    if not p.exists():
        print(f"WARNING: missing {p}")
        continue
    df = read_csv_local(fn)
    dfs[fn] = df
    print(f"Loaded {fn}: shape = {df.shape}")
list(dfs.keys())

Loaded lfs_monthly_variables.csv: shape = (388, 1831)
Loaded lfs_quarterly_variables.csv: shape = (124, 1831)
Loaded qual_survey_responses.csv: shape = (110, 4)
Loaded variable_names.csv: shape = (1830, 1)


['lfs_monthly_variables.csv',
 'lfs_quarterly_variables.csv',
 'qual_survey_responses.csv',
 'variable_names.csv']

In [41]:
# === Imports & tiny helpers ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

def raw_path(*parts) -> Path:
    return DATA_RAW.joinpath(*parts)

def processed_path(*parts) -> Path:
    p = DATA_PROCESSED.joinpath(*parts)
    p.parent.mkdir(parents=True, exist_ok=True)
    return p

def read_csv_local(filename: str, **kwargs) -> pd.DataFrame:
    return pd.read_csv(raw_path(filename), **kwargs)


In [42]:
# === Load data ===
dfs = {}
for fn in FILENAMES:
    p = raw_path(fn)
    if not p.exists():
        print(f"WARNING: missing {p}")
        continue
    df = pd.read_csv(raw_path(fn), encoding="cp1252")
    dfs[fn] = df
    print(f"Loaded {fn}: shape = {df.shape}")
list(dfs.keys())

Loaded lfs_monthly_variables.csv: shape = (388, 1831)
Loaded lfs_quarterly_variables.csv: shape = (124, 1831)
Loaded qual_survey_responses.csv: shape = (110, 4)
Loaded variable_names.csv: shape = (1830, 1)


['lfs_monthly_variables.csv',
 'lfs_quarterly_variables.csv',
 'qual_survey_responses.csv',
 'variable_names.csv']

In [43]:
dfs["lfs_quarterly_variables.csv"].describe().T


,count,unique,top,freq
Title,124,124,CDID,1
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Regular Pay,1,1,A2F9,1
AWE: Whole Economy Real Terms Year on Year three Month Growth (%): Seasonally Adjusted Regular Pay,1,1,A2FA,1
AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Regular Pay,99,51,454,5
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay,1,1,A3WV,1
...,...,...,...,...
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Sweden - Eurostat,108,43,6.9,5
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United Kingdom Eurostat,107,47,5.1,8
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United States,108,48,5.7,6
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Japan- Eurostat,108,32,4.7,8


In [44]:
dfs["lfs_monthly_variables.csv"].describe().T



,count,unique,top,freq
Title,388,388,CDID,1
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Regular Pay,280,73,1.7,12
AWE: Whole Economy Real Terms Year on Year three Month Growth (%): Seasonally Adjusted Regular Pay,278,68,1.5,14
AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Regular Pay,293,77,458,16
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay,280,81,1.7,10
...,...,...,...,...
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Sweden - Eurostat,335,52,7.8,16
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United Kingdom Eurostat,332,63,5.1,21
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United States,336,67,5.6,17
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Japan- Eurostat,335,35,3.4,21


In [45]:
# === Quick overview ===
for fn, df in dfs.items():
    print("\n---", fn, "---")
    display(df.head())
    display(df.describe(include="all").T)
    print("Nulls by column:")
    display(df.isna().sum())


--- lfs_monthly_variables.csv ---


,Title,AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Regular Pay,AWE: Whole Economy Real Terms Year on Year three Month Growth (%): Seasonally Adjusted Regular Pay,AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Regular Pay,AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay,AWE: Whole Economy Real Terms Year on Year Three Month Growth (%): Seasonally Adjusted Total Pay,AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Total Pay,Employment rate Canada (OECD) seasonally adjusted,Employment Rate Japan (OECD) seasonally adjusted,Employment Rate United States (OECD) seasonally adjusted,"Standardised ILO unemployment rates, seasonally adjusted, Romania - Eurostat","Standardised ILO unemployment rates, seasonally adjusted, Bulgaria - Eurostat",Standardised ILO Unemployment rates - Total EU,International Comparison Employment rates - Romania,International Comparison employment rates - Bulgaria,International Comparison - Employment rates - Total EU,International Comparison Employment Rates - Cyprus,International Comparison Employment Rates - Czech Republic,International Comparison Employment Rates - Estonia,International Comparison Employment Rates - Hungary,International Comparison Employment Rates - Latvia,International Comparison Employment Rates - Lithuania,International Comparison Employment Rates - Malta,International Comparison Employment Rates - Poland,International Comparison Employment Rates - Slovak Republic,International Comparison Employment Rates - Slovenia,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Cyprus Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Czech Republic Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Estonia Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Hungary Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Latvia Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Lithuania Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Malta Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Poland Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Slovak Republic Eurostat,STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Slovenia Eurostat,LFS: In employment: Aged 16-17: Not in full-time education: UK: All: 000s: SA,LFS: In employment: Aged 18-24: Not in full-time education: UK: All: 000s: SA,LFS: In employment: Aged 16-24: Not in full-time education: UK: All: 000s: SA,LFS: In employment: Aged 16-17: Not in full-time education: UK: Male: 000s: SA,LFS: In employment: Aged 18-24: Not in full-time education: UK: Male: 000s: SA,LFS: In employment: Aged 16-24: Not in full-time education: UK: Male: 000s: SA,LFS: In employment: Aged 16-17: Not in full-time education: UK: Female: 000s: SA,LFS: In employment: Aged 18-24: Not in full-time education: UK: Female: 000s: SA,LFS: In employment: Aged 16-24: Not in full-time education: UK: Female: 000s: SA,LFS: In employment: Aged 16-17: In full-time education: UK: All: 000s: SA,LFS: In employment: Aged 18-24: In full-time education: UK: All: 000s: SA,LFS: In employment: Aged 16-24: In full-time education: UK: All: 000s: SA,LFS: In employment: Aged 16-17: In full-time education: UK: Male: 000s: SA,LFS: In employment: Aged 18-24: In full-time education: UK: Male: 000s: SA,...,LFS: Economic inactivity rate: Aged 16-24: UK: All: %: NSA,LFS: Economic inactivity rate: Aged 16-24: UK: Male: %: NSA,LFS: Economic inactivity rate: Aged 16-24: UK: Female: %: NSA,LFS: Population aged 16 and over: Northern Ireland: All: Thousands: NSA,International Comparison Employment Rates Austria,International Comparison Employment Rates Belgium,Internatioonal Comparison Employment Rates Denmark,International Comparison Employment Rates Finland,International Comparison Employment Rates France,International Comparison Employment Rates Germany,International 

,count,unique,top,freq
Title,388,388,CDID,1
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Regular Pay,280,73,1.7,12
AWE: Whole Economy Real Terms Year on Year three Month Growth (%): Seasonally Adjusted Regular Pay,278,68,1.5,14
AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Regular Pay,293,77,458,16
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay,280,81,1.7,10
...,...,...,...,...
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Sweden - Eurostat,335,52,7.8,16
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United Kingdom Eurostat,332,63,5.1,21
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United States,336,67,5.6,17
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Japan- Eurostat,335,35,3.4,21


Nulls by column:


Title                                                                                                    0
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Regular Pay    108
AWE: Whole Economy Real Terms Year on Year three Month Growth (%): Seasonally Adjusted Regular Pay     110
AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Regular Pay                                95
AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay      108
                                                                                                      ... 
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED Sweden - Eurostat                               53
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United Kingdom Eurostat                         56
STANDARDISED ILO UNEMPLOYMENT RATES SEASONALLY ADJUSTED United States                                   52
STANDARDISED ILO UNEMPLOYMENT RATES S

...


--- variable_names.csv ---


,Title
0,AWE: Whole Economy Real Terms Year on Year Sin...
1,AWE: Whole Economy Real Terms Year on Year thr...
2,AWE: Whole Economy Real Terms Level (£): Seaso...
3,AWE: Whole Economy Real Terms Year on Year Sin...
4,AWE: Whole Economy Real Terms Year on Year Thr...


,count,unique,top,freq
Title,1830,1828,LFS: Unemployed over 6 and up to 12 months: UK...,2


Nulls by column:


Title    0
dtype: int64

In [ ]:
# === Quick overview ===
for fn, df in dfs.items():
    print('\n---', fn, '---')
    display(df.head())
    display(df.describe(include='all').T)
    print('Nulls by column:')
    display(df.isna().sum())


In [ ]:
# === Example plot ===
for fn, df in dfs.items():
    num_cols = df.select_dtypes(include='number').columns.tolist()
    if not num_cols:
        continue
    col = num_cols[0]
    plt.figure()
    df[col].dropna().hist()
    plt.title(f"{fn}: Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.show()
